In [2]:
from IPython.display import Image

In [ ]:
#multiple image inputs
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What are in these images? Is there any difference between them?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)
print(response.choices[0])

In [3]:
from dotenv import load_dotenv
from os import environ

load_dotenv("../../env")

# token_id = environ["modal-token-id"]
# token_secret = environ["modal-token-secret"]
openai_api_key = environ["OPENAI_API_KEY"]
assert sum(openai_api_key.encode('ascii')) == 4241, "OpenAI key is invalid"
print("OpenAI key is valid")

huggingface_api_key = environ["HUGGINGFACEHUB_API_TOKEN"]

# assert sum(token_id.encode('ascii')) == 2299, "Modal token ID is invalid"
# assert sum(token_secret.encode('ascii')) == 2075, "Modal token secret is invalid"
# print("Modal keys retrieved from Google drive are valid")


OpenAI key is valid


In [4]:
# to upload a local image
import base64
import requests


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "data/Enjoying-convertible-car.jpg"
Image(image_path)#, width=200, height=100)

# Getting the base64 string
base64_image = encode_image(image_path)


headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {openai_api_key}"
}

prompt = "What’s in this image?"

def prompt_image(prompt, image_path):
  base64_image = encode_image(image_path)
  
  payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  return response 

In [ ]:
print(prompt_image(prompt, image_path).json()["choices"][0]["message"]["content"])

The image shows two individuals in a blue convertible car. The person in the passenger seat is holding up a scarf or a piece of light fabric that is fluttering in the wind. Both individuals appear to be wearing sunglasses and are likely enjoying a sunny day, possibly out on a leisure drive. The environment around them includes trees which might be olive trees, and a small building structure in the background, suggesting a rural or Mediterranean setting. The scenery indicates a hilly or mountainous region, as you can see a landscape extending into the distance under a clear blue sky.


In [17]:
prompt1 = """ 
list all possible beliefs we can extract from this image, and express them in this format
<this thing or person> <action> <another thing><for this reason or purpose><in these conditions >... the last 2 fields are not mandatory
please keep the propositions with the <action> to leave <another thing> as an object/person name
I want general answers, not specific to this image.
For instance, for this image, I would expect something like:
<people><drive><convertible><because there is no risk of rain><in a sunny day>
not <the person in the driver seat><drive><the convertible><because he is happy><in a sunny day>
For <thing>, <person> and <action> be as generic and short as possible.
"""
ans1 = prompt_image(prompt1, image_path)
print(ans1.json()["choices"][0]["message"]["content"])

<People><wear><sunglasses><because of bright sunlight><in a sunny day>
<Vehicle><move><on a road><to transport individuals><in a rural area>
<Individual><wave><scarf><to express joy or freedom><while riding in a vehicle>
<Person><operate><vehicle><to travel from one place to another><on a clear day>
<Passenger><sit><in a vehicle><to accompany the driver or to reach a destination><during a journey>
<Clothing><flutter><in the wind><due to motion of the vehicle and outside air currents><while being held aloft>


In [21]:
prompt2 = """ 
list all possible beliefs we can extract from this image, and express them as a python dictionary with the following format:
{
    "condition": <conditions observed from the picture such as a sunny day, ie the conditions leading to the rest of the beliefs, such as the objective and action>,
    "objective": <the objective of the person or thing in the picture, after observing the conditions in the picture>,
    "subject": <the person or thing in the picture doing the 'action' to meet the 'objective', just one word if possible>,
    "action": <the action the person or thing in the picture is doing to meet the 'objective', expressed in one word with an optional preposition, such as 'drive to'>,
    "object": <the object of the action, expressed in one word if possible such as 'beach', NOT 'the beach'>,
}
Use an empty string when these fields are not available.
Do not return a belief if the subject and actions are not clearly identified.

I want general answers, not specific to this image.
For instance, for this image, I would expect something like:
{
    "condition": "sunny day",
    "objective": "enjoy the countryside",
    "subject": "people",
    "action": "drive",
    "object": "convertible",
}
not the following
{
    "condition": "in a sunny day",
    "objective": "because he enjoys the countryside",
    "subject": "the person in the driver seat",
    "action": "drive",
    "object": "the convertible",
}
For "subject", "action", "object", be as generic and short as possible.
Do this for all the beliefs you can extract from the image.
"""
ans1 = prompt_image(prompt2, image_path)
print(ans1.json()["choices"][0]["message"]["content"])

In [6]:
prompt3 = """ 
list all possible beliefs we can extract from this image, and express them in this format
<this thing or person> <action> <another thing><for this reason or purpose><in these conditions ><optional>... 
Please keep the propositions with the <action> to leave <another thing> as an object/person name

You can also use another format, when there is no action, using the verb 'to be', such as:
<this car><is><a convertible><because the roof can be removed>

To generate all possible beliefs, in an exhaustive way, look at all the objects, persons, background etc and try to find a relation between them following the formats above.
Do not output anything before the first '<' and after the last '>'.
Do not output the '<' or '>' characters.
"""
ans3 = prompt_image(prompt3, image_path)
print(ans3.json()["choices"][0]["message"]["content"])

this car is a convertible because the roof can be removed
the person in the passenger seat is waving a scarf for enjoyment in a moving car
the person in the driver's seat is driving this car to navigate the vehicle on the road
the people are wearing sunglasses for eye protection in sunny weather
the landscape includes trees indicative of a rural or semi-rural area
the car is on a road suggesting the action of traveling
the sky is clear suggesting it is fair weather
the structure in the background is a building possibly for residential or historical significance in a rural setting


In [5]:
prompt4 = """ 
Find all objects or people you can find this image and put them in a python list. 
Be exhaustive, do not miss any object or person, including in the background, the sky if there is one, through a window, behind people, even partially visible etc.
Just list one object at a time, do not include what they are doing or where they are, just the object / person name only.
For instance, "women wearing hats" is two objects "women" and "hats"
"""
ans4 = prompt_image(prompt4, image_path)
print(ans4.json()["choices"][0]["message"]["content"])

```python
objects_and_people = [
    "sky",
    "scarf",
    "woman",
    "sunglasses",
    "car",
    "trees",
    "man",
    "building",
    "landscape"
]
```


In [13]:
ans4.json()["choices"][0]["message"]["content"].split("\n")[2:-2]

['    "sky",',
 '    "scarf",',
 '    "woman",',
 '    "sunglasses",',
 '    "car",',
 '    "trees",',
 '    "man",',
 '    "building",',
 '    "landscape"']

In [28]:
objectPrompt="""
Find all the words between quotes in the following string and put them in a python string.
Just output the string without any comment, start with ' and end with ], nothing else.
Do not put quotes or double quotes around the words.
"""
requestMessage = objectPrompt + '\n' + ans4.json()["choices"][0]["message"]["content"]
objects = await client.chat.completions.create(model="gpt-4",  # previous models, even GPT3.5 didn't work that well
                                        messages=[{"role": "system", "content": "You are an expert in linguistics, semantic"},
                                                    {"role": "user", "content": requestMessage}
                                                    ]
                                        )
objects_in_image = objects.choices[0].message.content[1:-1].split(", ")
objects_in_image

"'sky, scarf, woman, sunglasses, car, trees, man, building, landscape'"

In [29]:
objects_in_image = objects.choices[0].message.content[1:-1].split(", ")
objects_in_image

['sky',
 'scarf',
 'woman',
 'sunglasses',
 'car',
 'trees',
 'man',
 'building',
 'landscape']

In [249]:
beliefs = [f"belief{i}: "+ans.replace('<','').replace('>',' ') for i,ans in enumerate(ans3.json()["choices"][0]["message"]["content"].split("\n"), 1)]
beliefs

['belief1: this car is a convertible because the roof can be removed',
 'belief2: the person in the passenger seat waves a scarf for joy in bright weather conditions',
 'belief3: the driver operates the car for transportation under clear sky conditions',
 'belief4: both individuals wear sunglasses for eye protection in sunny conditions',
 'belief5: the vegetation is typical of a Mediterranean environment because of the species and climate conditions present',
 'belief6: the car is on a dirt road for travel amidst a rural landscape ',
 'belief7: the building in the background is older for architectural style in a countryside setting',
 'belief8: the individuals are likely friends for engaging in a leisure activity together in a vehicle',
 'belief9: the scarf is colorful for aesthetic purposes in the daylight',
 'belief10: the car color is teal for its paint in daylight conditions']

In [110]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
token_list = enc.encode("LabLab is amazing!")
for token in token_list:
  print ("Token ", str(token), " decodes to ", enc.decode([token]))

print ("Number of tokens in corpus ", len(enc.encode("YOUR_INPUT_CHOICE")))

Token  30146  decodes to  Lab
Token  30146  decodes to  Lab
Token  374  decodes to   is
Token  8056  decodes to   amazing
Token  0  decodes to  !
Number of tokens in corpus  4


In [111]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def split_text (input_text):
  split_texts = sent_tokenize(input_text)
  return split_texts

[nltk_data] Downloading package punkt to /Users/jpb2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [112]:
def create_chunks(split_sents, max_token_len=2000):
  current_token_len = 0
  input_chunks = []
  current_chunk = ""
  for sents in split_sents:
    sent_token_len = len(enc.encode(sents))
    if (current_token_len + sent_token_len) > max_token_len:
      input_chunks.append(current_chunk) # we reached the limit, add new chunk 'current chunk', then reset it
      current_chunk = ""
      current_token_len = 0
    current_chunk = current_chunk + sents
    current_token_len = current_token_len + sent_token_len
  if current_chunk != "":
    input_chunks.append(current_chunk)
  return input_chunks

In [ ]:
split_sents = split_text(beliefs)
input_chunks = create_chunks(split_sents, max_token_len=2000)

In [15]:
from openai import AsyncOpenAI

client = AsyncOpenAI()

In [250]:
instructPrompt = """
You are an expert in linguistics, semantic and you are trying to format the beliefs passed to you into a format that can be stored in a knowledge graph.
The beliefs start with the word 'belief1:', 'belief2:' etc and are separated by a new line.

Rewrite every belief and express them as a python dictionary with the following format:
{
    "condition": <conditions observed from the picture such as a sunny day, ie the conditions leading to the rest of the beliefs, such as the objective and action>,
    "objective": <the objective of the person or thing in the picture, after observing the conditions in the picture>,
    "subject": <the person or thing in the picture doing the 'action' to meet the 'objective', just one word if possible>,
    "action": <the action the person or thing in the picture is doing to meet the 'objective', expressed in one word with an optional preposition, such as 'drive to'>,
    "object": <the object of the action, expressed in one word if possible such as 'beach', NOT 'the beach'>
}
Use an empty string when a field is not available.
Do not return a belief if the subject and actions are not clearly identified.

I want general answers, not specific to an image.
For instance, I would expect something like:
{
    "condition": "sunny day",
    "objective": "enjoy the countryside",
    "subject": "people",
    "action": "drive",
    "object": "convertible"
}
not the following
{
    "condition": "in a sunny day",
    "objective": "because he enjoys the countryside",
    "subject": "the person in the driver seat",
    "action": "drive",
    "object": "the convertible"
}
For "subject", "action", "object", be as generic and short as possible.

When the belief does not have an action, but instead use a verb such as 'to be' for instance, then put the verb in the <action anyway>, 
and use the field 'object' to describe what the object or person is or is made of or anything else that the verb describes.

Use infinitive verbs, without the 'to', ie 'be' instead of 'is'.  
"""

In [251]:
requestMessages = [instructPrompt + '\n' + belief for belief in beliefs]
chatOutputs = []
for request in requestMessages: 
  chatOutput = await client.chat.completions.create(model="gpt-4",  # previous models, even GPT3.5 didn't work that well
                                            messages=[{"role": "system", "content": "You are an expert in linguistics, semantic"},
                                                      {"role": "user", "content": request}
                                                      ]
                                            )
  chatOutputs.append(chatOutput)

In [252]:
formatted_beliefs = {}
for belief in chatOutputs:
  b = json.loads(belief.choices[0].message.content)
  formatted_beliefs[b["subject"]] = b
  print(b)

{'condition': 'roof can be removed', 'objective': '', 'subject': 'car', 'action': 'be', 'object': 'convertible'}
{'condition': 'bright weather conditions', 'objective': 'express joy', 'subject': 'person', 'action': 'wave', 'object': 'scarf'}
{'condition': 'clear sky', 'objective': 'transportation', 'subject': 'driver', 'action': 'operate', 'object': 'car'}
{'condition': 'sunny conditions', 'objective': 'eye protection', 'subject': 'individuals', 'action': 'wear', 'object': 'sunglasses'}
{'condition': 'Mediterranean environment', 'objective': '', 'subject': 'vegetation', 'action': 'be', 'object': 'typical'}
{'condition': 'rural landscape', 'objective': 'travel', 'subject': 'car', 'action': 'be', 'object': 'on a dirt road'}
{'condition': 'countryside setting', 'objective': '', 'subject': 'building', 'action': 'be', 'object': 'older'}
{'condition': 'engaging in leisure activity together', 'objective': '', 'subject': 'individuals', 'action': 'be', 'object': 'friends'}
{'condition': 'daylig

In [ ]:
# I setup a free Neo4J instance on Aura, and I'm using the Python driver to connect to it
import json, configparser
from py2neo import Relationship

Neo4j_config = configparser.ConfigParser()
Neo4j_config.read('neo4j_config.ini')
Neo4j_config['DEFAULT']["pw"] = environ['NEO4J_AURA_PW']

g = Neo4jGraph(showstatus=True, **Neo4j_config['DEFAULT'])

# in case of error, check your config.ini file

# Instructions to see the graph in Neo4J Browser
# go to 
# https://workspace-preview.neo4j.io/workspace/query?ntid=auth0%7C631bb4216f68981ab949290b
# run the cypher query: 
# MATCH (n) RETURN n     to see all nodes


Graph Connected


In [211]:
g.deleteAllNodes(DEBUG=True)
assert g.nodesNb == 0
print("Graph has been cleaned")

All nodes have been deleted!
Graph has been cleaned


In [243]:
g.deleteAllNodes(DEBUG=True)
assert g.nodesNb == 0
print("Graph has been cleaned")

for b in chatOutputs:
  
  belief = json.loads(b.choices[0].message.content)
  
  name=belief.pop("subject")
  object = belief.pop("object")
  relation = belief.pop("action")
  print('\n'.join([name, relation, object, belief['objective'], belief['condition']]))
  
  try:
    subject1  = createNode(g, name,
                          user_id='JPB',
                          display_name=name,
                          labels_constraints=name,  # we don't create the same object/subject twice
                          # properties_constraints=('user_id',), 
                          creation_timestamp = Now(),
                          DEBUG=True)  # we can have the same subject appear several times
    
  except:
    print(f"node not created for {name}")
  
  try:
    object1  = createNode(g, object,
                          user_id='JPB',
                          display_name=object,
                          labels_constraints=object,  
                          properties_constraints=('user_id',), 
                          creation_timestamp = Now(),
                          DEBUG=True,
                          **belief,
                          )
  except:
    print(f"node not created for {object}")
    
  try:
    relat1 = createRelation(g, subject1, 
                            object1, 
                            relation,
                            DEBUG=True,
                            #synonyms='has imagery',  # for demo, relations can have properties as well
                            allow_duplicates=True,   # Neo4J allows several identical relations between 2 nodes, but we don't want that
                            counting=True)
  except:
    print(f"relation not created for 'to {relation}'")
  
  print("="*50)
    
print("Nb of nodes =", g.nodesNb)


All nodes have been deleted!
Graph has been cleaned
car
be
convertible

the roof can be removed
New node ('car',) created with ID = 13
New node ('convertible',) created with ID = 14
Relationship 'be' created between nodes 13 and node 14
person
wave
scarf
express joy
bright weather conditions
New node ('person',) created with ID = 0
New node ('scarf',) created with ID = 1
Relationship 'wave' created between nodes 0 and node 1
driver
operate
car
transportation
clear sky conditions
New node ('driver',) created with ID = 2
Node ('car',) already exists with ID = 13
Relationship 'operate' created between nodes 2 and node 13
individuals
wear
sunglasses
eye protection
sunny conditions
New node ('individuals',) created with ID = 3
New node ('sunglasses',) created with ID = 4
Relationship 'wear' created between nodes 3 and node 4
vegetation
be
typical

Mediterranean environment
New node ('vegetation',) created with ID = 5
New node ('typical',) created with ID = 6
Relationship 'be' created betwee

In [ ]:
import neo4jupyter
neo4jupyter.init_notebook_mode()

<IPython.core.display.Javascript object>

This graph represents the online Neo4J graph database.
It's dynamic, you can move the nodes around.

In [248]:
# see https://github.com/merqurio/neo4jupyter/blob/master/README.md
neo4jupyter.draw(g.graph, {"user_id": "JPB"})
